In [1]:
%load_ext autoreload
%autoreload now

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"
# os.environ["WORLD_SIZE"] = "1"

# Download and convert data

In [1]:
from tqdm import tqdm

from document_segmentation.pagexml.annotations.generale_missiven import GeneraleMissiven
from document_segmentation.settings import (
    GENERALE_MISSIVEN_DOCUMENT_DIR,
    GENERALE_MISSIVEN_SHEET,
)

N = None

GENERALE_MISSIVEN_DOCUMENT_DIR.mkdir(parents=True, exist_ok=True)

sheet = GeneraleMissiven(GENERALE_MISSIVEN_SHEET)

existing_docs = {
    path.stem
    for path in GENERALE_MISSIVEN_DOCUMENT_DIR.glob("*.json")
    if path.is_file()
}

for document in tqdm(
    sheet.to_documents(n=N, skip_ids=existing_docs),
    total=(N or len(sheet)) - len(existing_docs),
    desc="Writing documents",
    unit="doc",
):
    document_file = GENERALE_MISSIVEN_DOCUMENT_DIR / f"{document.id}.json"

    with document_file.open("xt") as f:
        f.write(document.model_dump_json())
        f.write("\n")

Writing documents:  12%|█▏        | 114/914 [04:00<30:03,  2.25s/doc] 

Skipping row with inventory number 1171 due to reason: 'Niet gedigitaliseerd.'


Writing documents:  36%|███▋      | 332/914 [14:04<17:26,  1.80s/doc]  

Skipping inventory '1684' due to error: 404 Client Error: Not Found for url: https://hucdrive.huc.knaw.nl/HTR/obp-v2-pagexml-leon-metadata-trimmed-2023-11/1684.zip


Writing documents:  36%|███▋      | 333/914 [14:05<14:28,  1.50s/doc]

...

Writing documents:  37%|███▋      | 334/914 [14:05<12:00,  1.24s/doc]

Skipping inventory '1684' due to error: 404 Client Error: Not Found for url: https://hucdrive.huc.knaw.nl/HTR/obp-v2-pagexml-leon-metadata-trimmed-2023-11/1684.zip


Writing documents:  42%|████▏     | 386/914 [17:16<30:44,  3.49s/doc]  

Skipping inventory '1887' due to error: 404 Client Error: Not Found for url: https://hucdrive.huc.knaw.nl/HTR/obp-v2-pagexml-leon-metadata-trimmed-2023-11/1887.zip


Writing documents:  44%|████▍     | 405/914 [18:37<32:50,  3.87s/doc]

# Load Data

In [4]:
%autoreload now

In [5]:
TRAINING_DATA = 0.8

In [6]:
from document_segmentation.model.dataset import PageDataset
from document_segmentation.settings import MIN_REGION_TEXT_LENGTH

dataset = PageDataset.from_dir(GENERALE_MISSIVEN_DOCUMENT_DIR).remove_short_regions(
    MIN_REGION_TEXT_LENGTH
)
len(dataset)

Reading JSON files: 100%|██████████| 909/909 [01:25<00:00, 10.65file/s]


191146

In [7]:
dataset._class_counts()

Counter({<Label.IN: 2>: 189343, <Label.BEGIN: 1>: 905, <Label.END: 3>: 898})

In [8]:
dataset.class_weights()

[210.97792494481237, 1.0095170694608755, 212.6206896551724, 191146.0]

In [9]:
dataset[5000]

Page(label=<Label.IN: 2>, regions=[Region(id='region_ae02e415-afa1-48f8-aeef-2ad2960df7fc_8', types=(<RegionType.TEXT_REGION: 'text_region'>, <RegionType.MARGINALIA: 'marginalia'>, <RegionType.PHYSICAL_STRUCTURE_DOC: 'physical_structure_doc'>, <RegionType.PAGEXML_DOC: 'pagexml_doc'>), coordinates=((451, 882), (455, 886), (455, 907), (460, 911), (460, 915), (468, 923), (468, 927), (472, 927), (480, 936), (501, 936), (505, 940), (551, 940), (555, 936), (580, 936), (584, 931), (613, 931), (617, 927), (675, 927), (679, 931), (725, 931), (729, 936), (870, 936), (874, 931), (886, 931), (894, 923), (899, 923), (907, 915), (911, 915), (915, 911), (923, 911), (927, 907), (940, 907), (944, 902), (948, 902), (956, 894), (961, 894), (965, 890), (977, 890), (981, 886), (998, 886), (1002, 882), (1027, 882), (1031, 878), (1048, 878), (1056, 869), (1060, 869), (1060, 861), (1064, 857), (1064, 844), (1068, 840), (1068, 828), (1072, 824), (1072, 820), (1077, 816), (1077, 807), (1081, 803), (1081, 766), 

In [10]:
split = int(len(dataset) * TRAINING_DATA)

training_data = dataset[:split]
training_data._class_counts()

Counter({<Label.IN: 2>: 151442, <Label.BEGIN: 1>: 740, <Label.END: 3>: 734})

In [11]:
test_data = dataset[split:]
test_data._class_counts()

Counter({<Label.IN: 2>: 37901, <Label.BEGIN: 1>: 165, <Label.END: 3>: 164})

# Train Model

In [12]:
import torch

BATCH_SIZE = 32
EPOCHS = 3
WEIGHTS = torch.Tensor(dataset.class_weights())  # For an imbalanced dataset

In [13]:
%autoreload now

In [14]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

/home/carstens/workspace/.venv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [15]:
tagger._device

'cuda'

In [16]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
      )
      (_region_type): Embedding(9, 16)
      (_linear): Linear(in_features=784, out_features=128, bias=True)
    )
    (_rnn): LSTM(128, 128, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=256, out_features=128, bias=True)
  )
  (_rnn): LSTM(128, 64, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=128, out_features=4, bias=True)
  (_softmax): Softmax(dim=1)
)

In [17]:
tagger.train_(training_data, EPOCHS, BATCH_SIZE, WEIGHTS.to(tagger._device))

313batch [06:34,  1.26s/batch]                        


[Loss:	1.624]


313batch [00:14, 21.18batch/s]                        


[Loss:	1.557]


313batch [00:14, 21.03batch/s]                        

[Loss:	1.580]


# Evaluate Model

In [18]:
import csv
import sys

from torcheval.metrics import (
    MulticlassAccuracy,
    MulticlassF1Score,
    MulticlassPrecision,
    MulticlassRecall,
)
from tqdm import tqdm

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout,
    fieldnames=("Predicted", "Actual", "Page ID", "Text", "Scores"),
    delimiter="\t",
)

writer.writeheader()

accuracy = MulticlassAccuracy(num_classes=len(Label))
precision = MulticlassPrecision(average=None, num_classes=len(Label))
recall = MulticlassRecall(average=None, num_classes=len(Label))
f1_score = MulticlassF1Score(average=None, num_classes=len(Label))

for batch in tqdm(
    test_data.batches(BATCH_SIZE), total=len(test_data) / BATCH_SIZE, unit="batch"
):
    predicted = tagger(batch)
    labels = batch.labels()

    _labels = torch.Tensor([label.value for label in labels]).to(int)
    accuracy.update(predicted, _labels)
    precision.update(predicted, _labels)
    recall.update(predicted, _labels)
    f1_score.update(predicted, _labels)

    for page, pred, label in zip(batch.pages, predicted, labels):
        pred_label = Label(pred.argmax().item())
        if pred_label != Label.IN or label != Label.IN:
            writer.writerow(
                {
                    "Predicted": pred_label.name,
                    "Actual": label.name,
                    "Page ID": page.doc_id,
                    "Text": page.text(delimiter="; ")[:50],
                    "Scores": str(pred.tolist()),
                }
            )

Predicted	Actual	Page ID	Text	Scores


  0%|          | 1/1194.6875 [00:01<25:10,  1.27s/batch]

BEGIN	IN	NL-HaNA_1.04.02_3655_0704.jpg	aan bevolene voor; sigtigheijd hier om; taend; gel	[0.49982118606567383, 0.10057077556848526, 0.39904433488845825, 0.0005636932328343391]
BEGIN	IN	NL-HaNA_1.04.02_3655_0705.jpg	den aan de vesting; werken te besteeden; gen nieuw	[0.5101383924484253, 0.08944611251354218, 0.40011852979660034, 0.00029701655148528516]
BEGIN	IN	NL-HaNA_1.04.02_3655_0706.jpg	„durven Cadjang en 8402½ lb: Car„; „waat Ceijlon a	[0.5108591914176941, 0.08565764874219894, 0.4032462537288666, 0.00023693002003710717]
BEGIN	IN	NL-HaNA_1.04.02_3655_0707.jpg	aanmerking op de; veragtende staat; ment:t; van 17	[0.5149756073951721, 0.08391320705413818, 0.40088680386543274, 0.00022434852144215256]
BEGIN	IN	NL-HaNA_1.04.02_3655_0708.jpg	egter op hope van; beteaftchap in tijds; vervolg; 	[0.509358823299408, 0.08407554030418396, 0.4063514769077301, 0.00021412863861769438]
BEGIN	IN	NL-HaNA_1.04.02_3655_0709.jpg	remarqie op der; min s eijsch uit Neder„; land; on	[0.5147278308868408, 0.08296

  0%|          | 2/1194.6875 [00:02<22:35,  1.14s/batch]

BEGIN	IN	NL-HaNA_1.04.02_3655_0736.jpg	goede scrikking der; min: omtrend de geld; ders.; 	[0.4990115165710449, 0.10066745430231094, 0.39975517988204956, 0.0005658891168422997]
BEGIN	IN	NL-HaNA_1.04.02_3655_0737.jpg	aanmerkingen op de; veragtende staat van; dit gouv	[0.5127236247062683, 0.08901804685592651, 0.3979533314704895, 0.0003049642837140709]
BEGIN	IN	NL-HaNA_1.04.02_3655_0738.jpg	178 2/3. op sig selve beschouwd gebleeven; is een 	[0.5151724219322205, 0.08541471511125565, 0.39916759729385376, 0.0002453007036820054]
BEGIN	IN	NL-HaNA_1.04.02_3655_0739.jpg	koat verslag van de; oorlog zanken; december 1783.	[0.5092990398406982, 0.08475687354803085, 0.4057197868824005, 0.0002242958580609411]
BEGIN	IN	NL-HaNA_1.04.02_3655_0740.jpg	qualificatie van de; vreemde scheepen Ni; „den en 	[0.5115519762039185, 0.0839945524930954, 0.40422680974006653, 0.0002267389791086316]
BEGIN	IN	NL-HaNA_1.04.02_3655_0741.jpg	bewesene adsistentee; aan een engelsch schip; geap	[0.4987689256668091, 0.0859220251

...

BEGIN	IN	NL-HaNA_1.04.02_3419_0178.jpg	twee der bevorder„; den zijn overle„; den; kiezing	[0.49463769793510437, 0.1046857014298439, 0.4000813066959381, 0.0005952311912551522]
BEGIN	IN	NL-HaNA_1.04.02_3419_0179.jpg	gen aan de Caab„; se ministers is in; Copia naam.;	[0.5033628940582275, 0.09129293262958527, 0.4050382673740387, 0.0003058999718632549]
BEGIN	IN	NL-HaNA_1.04.02_3419_0180.jpg	Ian Paardeko„; per Daniel; en paulus; en de laatse	[0.5155781507492065, 0.084412582218647, 0.39977672696113586, 0.00023255389533005655]
BEGIN	IN	NL-HaNA_1.04.02_3419_0181.jpg	Thuijs ter Meije; aangebragt; lonse minis„; ters v	[0.5062285661697388, 0.08540856838226318, 0.4081442058086395, 0.00021870926138944924]
BEGIN	IN	NL-HaNA_1.04.02_3419_0182.jpg	de scheeps overheeden; is aangekundigt; dat zij 't	[0.49777576327323914, 0.08731397241353989, 0.4146905839443207, 0.00021970363741274923]
BEGIN	IN	NL-HaNA_1.04.02_3419_0183.jpg	tusschende 50 en; 60'00000 lb: zullen.; voldaan wo	[0.5030777454376221, 0.084351502

  3%|▎         | 31/1194.6875 [00:40<29:21,  1.51s/batch]

BEGIN	IN	NL-HaNA_1.04.02_3419_0210.jpg	Banda 'T de; torierende; staat beschree„; ven; get	[0.4963412582874298, 0.10205525904893875, 0.4010288715362549, 0.0005746662500314415]
BEGIN	IN	NL-HaNA_1.04.02_3419_0211.jpg	drukking van; over het verval; van de perke„; nier	[0.5067297220230103, 0.09012532234191895, 0.40283405780792236, 0.0003108519595116377]
BEGIN	IN	NL-HaNA_1.04.02_3419_0212.jpg	oorzaken, dewijl daar door, ook de thans zo; despe	[0.48937124013900757, 0.09049609303474426, 0.4198531210422516, 0.0002795369946397841]
BEGIN	IN	NL-HaNA_1.04.02_3419_0213.jpg	consideratien; over de gunst; bewijzingen; der het	[0.482010155916214, 0.10205921530723572, 0.4153747260570526, 0.0005559048149734735]
BEGIN	IN	NL-HaNA_1.04.02_3419_0214.jpg	hoe veel de„; zelve s Jaars; in gelde belo„; pen; 	[0.5125778317451477, 0.08394184708595276, 0.4032641053199768, 0.00021617345919366926]
BEGIN	IN	NL-HaNA_1.04.02_3419_0215.jpg	nadere con„; sideratie over; de prijs verhoging; h	[0.5154523253440857, 0.0833742320

  3%|▎         | 32/1194.6875 [00:40<24:47,  1.28s/batch]

BEGIN	IN	NL-HaNA_1.04.02_3419_0242.jpg	wat met de; vrouwe geer„; truijda nog; vervoert; s	[0.4954198896884918, 0.1016128659248352, 0.40239349007606506, 0.0005737515748478472]
BEGIN	IN	NL-HaNA_1.04.02_3419_0243.jpg	ordre tot 't ver„; of nelijnaber; Ceylon; voor sou	[0.5048902034759521, 0.09008461982011795, 0.40471044182777405, 0.0003147496609017253]
BEGIN	IN	NL-HaNA_1.04.02_3419_0244.jpg	bepaalde zeil„; dag van sourat„s; te; de overheden	[0.500177264213562, 0.08850952982902527, 0.4110248386859894, 0.0002884180867113173]
BEGIN	IN	NL-HaNA_1.04.02_3419_0245.jpg	't schip De pa„; triot naar de; Caab aange„; legt;	[0.48084691166877747, 0.10349668562412262, 0.41505348682403564, 0.0006029442884027958]
BEGIN	IN	NL-HaNA_1.04.02_3419_0246.jpg	slegte verwag Bantam is ons door de bedienen den b	[0.5087472796440125, 0.08584626019001007, 0.4051641523838043, 0.00024229136761277914]
BEGIN	IN	NL-HaNA_1.04.02_3419_0247.jpg	en Isaac Reynst, den ondergetekenden Batavia; gouv	[0.4955953359603882, 0.088992647

In [20]:
writer = csv.DictWriter(
    sys.stdout,
    fieldnames=["Metric", "Average"] + [label.name for label in Label],
    delimiter="\t",
)
writer.writeheader()

for metric in (precision, recall, f1_score):
    scores = {
        label.name: f"{score:.4f}"
        for label, score in zip(Label, metric.compute().tolist())
    }
    writer.writerow(
        {"Metric": metric.__class__.__name__, "Average": str(metric.average)} | scores
    )

print(f"Accuracy ({accuracy.average} average):\t{accuracy.compute().item():.4f}")

        [3]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.


Metric	Average	BEGIN	IN	END	OUT
MulticlassPrecision	None	0.0051	0.0000	0.0000	0.0000
MulticlassRecall	None	0.8333	0.0000	0.0000	0.0000
MulticlassF1Score	None	0.0102	0.0000	0.0000	0.0000
Accuracy (micro average):	0.0050
